# Set Up

In [1]:
from pymongo import MongoClient
import pandas as pd
import bson

# How many Canadian residents(figure out who are Canadian residents by yourself) 
# reviewed the business “Mon Ami Gabi” in last 1 year?

In [2]:
client = MongoClient()
dblist = client.list_database_names()
db = client.yelp


# Find Reviews for MAG in the last year

### Find MAG business_id

In [3]:
mon_ami_gabi = db.business.find_one({"name": "Mon Ami Gabi"}, {"name": 1, "business_id": 1, "_id": 1, "city": 1})
mag_business_id = mon_ami_gabi["business_id"]
mag_business_id

'4JNXUYY8wbaaDmk3BPzlWw'

### Find MAG reviews in 2018

In [4]:
review_query = {"business_id": mag_business_id, "date": {"$gt": "2018-01-01 00:00:00", "$lt": "2019-01-01 00:00:00"}}
fields_wanted = {"business_id": 1, "user_id": 1, "review_id": 1, "date": 1}
mag_reviews = db.review.find(review_query, fields_wanted)
df_mag_reviews = pd.DataFrame(list(mag_reviews))
df_mag_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 964 entries, 0 to 963
Data columns (total 5 columns):
_id            964 non-null object
review_id      964 non-null object
user_id        964 non-null object
business_id    964 non-null object
date           964 non-null object
dtypes: object(5)
memory usage: 37.8+ KB


# Find users who have reviewed MAG

In [5]:
mon_ami_gabi_reviewers = df_mag_reviews['user_id'].unique()

In [6]:
df_mag_reviewers = pd.DataFrame(mon_ami_gabi_reviewers, columns=['user_id'])
df_mag_reviewers.head()
df_mag_reviewers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 951 entries, 0 to 950
Data columns (total 1 columns):
user_id    951 non-null object
dtypes: object(1)
memory usage: 7.6+ KB


# Find all the reviews for each user

In [7]:
review_query = {"business_id": mag_business_id, "date": {"$gt": "2018-01-01 00:00:00", "$lt": "2019-01-01 00:00:00"}}
project = {"user_id": 1, "review_id": 1, "_id": 0}
group = {"_id": "$user_id"}
lookup = {"from": "review", "localField": "user_id", "foreignField": "user_id", "as": "reviews_written"}
pipeline = [
    {'$match': review_query}, 
    {'$project': project},
    {'$group': group},
#     {'$lookup': lookup}
    
]
cursor = db.review.aggregate(pipeline)
lst = list(cursor)
lst[0]
# len(lst)
# len(lst) #964 without group, 951 with group


{'_id': '-7M6CdMaGgBI82-yJvfCsw'}

In [8]:
review_query = {"business_id": mag_business_id, "date": {"$gt": "2018-01-01 00:00:00", "$lt": "2019-01-01 00:00:00"}}
project = {"user_id": 1, "review_id": 1, "_id": 0}
group = {"_id": "$user_id"}
pipeline = [
    {'$match': review_query}, 
    {'$project': project},
    {'$group': group},    
]
cursor = db.review.aggregate(pipeline)
# len(lst)

In [14]:
mag_reviewers = []
for user in cursor:
    print(user)
    reviews = list(db.review.find({"user_id": user}, {"review_id": 1, "user_id": 1, "business_id": 1}))
    dic = {user: reviews}
    mag_reviewers.append(dic)

    

In [10]:
# db.myCollection.find().forEach(bson.Code( '''
#     function(u) { 
#        u.forSong = self.request.db.song.find_one({}, {'_id': 1})
#        self.request.db.save(u)
#      }'''))



# db.myCollection.find().forEach(bson.Code( '''
#     function(u) { 
#        u.forSong = self.request.db.song.find_one({}, {'_id': 1})
#        self.request.db.save(u)
#      }'''))
#
# db.books.find().forEach(
#     function (newBook) {
#         newBook.category = db.categories.findOne( { "_id": newBook.category } );
#         newBook.lendings = db.lendings.find( { "book": newBook._id  } ).toArray();
#         newBook.authors = db.authors.find( { "_id": { $in: newBook.authors }  } ).toArray();
#         db.booksReloaded.insert(newBook);
#     }
# );


# FIND CANADIAN BUSINESS ID

In [11]:
provinces = ["ON", "QC", "NS", "NB", "MB", "BC", "PE", "SK", "AB", "NL", "NT", "YT", "NU"]
cad_biz_query = {"state": {"$in": provinces}}
# biz_fields = {"business_id": 1, "name": 1, "city": 1, "state": 1}
biz_fields = {"business_id": 1}

canadian_business_ids = db.business.find(cad_biz_query, biz_fields)

df = pd.DataFrame(list(canadian_business_ids))
canadian_business_id_list = list(df.business_id)
set_canadian_business_id = set(canadian_business_id_list)

In [12]:
len(set_canadian_business_id)

50644

In [2]:
 50644 / 1637138 * 100

3.0934472231418484